In [ ]:
!pip install -q numpy pandas transformers torch torch_geometric scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.3 MB/s eta 0:00:00


In [ ]:
import torch

torch.manual_seed(43)
torch.cuda.manual_seed_all(43)

In [ ]:
import pandas as pd

df = pd.read_csv('good_train.csv')

In [ ]:
df2 = pd.read_csv('train.csv')

In [ ]:
count = 0
for i in df2['text'].tolist():
    if i in df['text'].tolist():
        count += 1
count

2768

In [ ]:
# Load model directly
from transformers import DebertaTokenizer, DebertaModel
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

In [ ]:
punct = {'?', '!'}
def clean_text(text):
    return ''.join(ch for ch in text if ch.isalnum() or ch in punct or ch == ' ')

clean_text("hi hS!,i nd ")

'hi hS!i nd '

In [ ]:
encoded = df['text'].apply(lambda x: tokenizer.encode_plus(
    x,
    max_length=64,
    padding='max_length',
    truncation=True,
    return_attention_mask=True
))
# df['text'].apply(clean_text)

df['input_ids'] = encoded.apply(lambda x: x['input_ids'])
df['attention_mask'] = encoded.apply(lambda x: x['attention_mask'])

In [ ]:
emotions = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
emotions2 = ['anger', 'fear', 'joy', 'sadness', 'surprise']

In [ ]:
import numpy as np
import torch
import math

cooccurance_matrix = np.zeros((5,5))

for idx, emotion in enumerate(emotions2):
    emotion_total = len(df[df[emotion] == 1])
    for idx2, emotion2 in enumerate(emotions2):
        if idx == idx2:
            cooccurance_matrix[idx][idx2] = len(df[(df[emotion] == 1) & (df[emotion2] == 1)]) / emotion_total
        else:
            cooccurance_matrix[idx][idx2] = (len(df[(df[emotion] == 1) & (df[emotion2] == 1)]) / emotion_total) * .9

degree = np.sum(cooccurance_matrix, axis=1)
d_inv_sqrt = np.diag(np.power(degree, -0.5))
d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0
d_inv_sqrt[np.isnan(d_inv_sqrt)] = 0

normalize_cooccurance_matrix = d_inv_sqrt @ cooccurance_matrix @ d_inv_sqrt
normalize_cooccurance_matrix = torch.tensor(normalize_cooccurance_matrix, dtype=torch.float32)

In [ ]:
normalize_cooccurance_matrix

tensor([[0.4195, 0.3036, 0.0103, 0.1856, 0.1343],
        [0.0627, 0.5265, 0.0354, 0.1903, 0.1625],
        [0.0051, 0.0845, 0.7039, 0.0362, 0.1205],
        [0.0704, 0.3492, 0.0278, 0.4739, 0.0967],
        [0.0533, 0.3119, 0.0968, 0.1012, 0.4693]])

In [ ]:
weights = []

for emotion in emotions2:
    weights.append(len(df[df[emotion] == 1]))

weights = [sum(weights) / w for w in weights]

In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split

class TweetsDataset(Dataset):

    def __init__(self, df, labels):
        self.df = df.copy()
        self.labels = labels

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx, :]

        return {
            'input_ids': torch.tensor(row['input_ids']),
            'attention_mask': torch.tensor(row['attention_mask']),
            'labels': torch.tensor(row[self.labels].tolist(), dtype=torch.float32)
        }

dataset = TweetsDataset(df, emotions2)

train_size = int(len(df) * .8)
val_size = len(df) - train_size

train_dataset, val_dataset = random_split(
    dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

train_dataloader = DataLoader(train_dataset, batch_size=16)
val_dataloader = DataLoader(val_dataset, batch_size=16)

full_dataloader = DataLoader(dataset, batch_size=16)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class TweetEmotionClassifier(nn.Module):

    def __init__(self, adj_matrix):
        super().__init__()

        self.bert = DebertaModel.from_pretrained("microsoft/deberta-base")
        self.adj_matrix = adj_matrix

        self.emotions_embeddings = torch.eye(5).to(device)
        self.w1 = nn.Parameter(torch.zeros((5, 768)))
        nn.init.kaiming_uniform_(self.w1, mode='fan_in', nonlinearity='relu')

        # self.w2 = nn.Parameter(torch.zeros((200, 768)))
        # nn.init.kaiming_uniform_(self.w2, mode='fan_in', nonlinearity='relu')

        self.dropout = nn.Dropout(0.4)

    def forward(self, input_ids, attention_mask):
        h = self.adj_matrix @ self.emotions_embeddings @ self.w1
        h = self.dropout(h)
        h = F.relu(h)

        # h = self.adj_matrix @ h @ self.w2
        # h = self.dropout(h)
        # h = F.relu(h)

        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]

        return cls_output @ h.t()

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class NonGraph(nn.Module):

    def __init__(self):
        super().__init__()

        self.bert = DebertaModel.from_pretrained('microsoft/deberta-base')
        self.dropout = nn.Dropout(0.3)

        # for param in self.bert.parameters():
        #     param.requires_grad = False

        self.linear = nn.Linear(768, 200)
        self.linear2 = nn.Linear(200, 5)


    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]

        h = self.dropout(cls_output)
        h = self.linear(h)
        h = F.relu(h)

        h = self.dropout(h)
        h = self.linear2(h)
        return h

In [ ]:
normalize_cooccurance_matrix = normalize_cooccurance_matrix.to(device)
# model = TweetEmotionClassifier(normalize_cooccurance_matrix)
model = NonGraph()
model = model.to(device)

# optimizer = torch.optim.AdamW([
#     {
#         "params": model.bert.encoder.layer[:6].parameters(),
#         "lr": 1e-5,
#     },
#     {
#         "params": model.bert.encoder.layer[6:12].parameters(),
#         "lr": 3e-5,
#     },
#     {'params': [model.w1], 'lr': 1e-3}
# ])


optimizer = torch.optim.AdamW([
    {'params': model.bert.parameters(), 'lr': 2e-5},
    {'params': list(model.linear.parameters()), 'lr': 1e-4},
    {'params': list(model.linear2.parameters()), 'lr': 1e-4}

])

loss_fn = nn.BCEWithLogitsLoss(weight=torch.tensor(weights).to(device))

In [ ]:
from sklearn.metrics import f1_score

for epoch in range(12):
    model.train()

    # for param in model.bert.encoder.layer[12 - epoch - 1].parameters():
    #         param.requires_grad = True

    for idx, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        pred = model(input_ids, attention_mask)

        loss = loss_fn(pred, labels)

        loss.backward()
        optimizer.step()

        # if idx % 50 == 0:
        #     print(loss.item())

        model.eval()
        all_preds = []
        all_labels = []

    with torch.no_grad():
        for idx, batch in enumerate(val_dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels']

            preds = model(input_ids, attention_mask)
            preds = preds.cpu().tolist()
            preds = [1 if i > 0.5 else 0 for b in preds for i in b]

            labels = [j for i in labels for j in i]
            all_preds.extend(preds)
            all_labels.extend(labels)

    s = f1_score(all_labels, all_preds, average='macro')
    print(f'f1_score={s}')


f1_score=0.7858910203702907
f1_score=0.8162635441743955
f1_score=0.8193404516512169
f1_score=0.8188486823300749
f1_score=0.7977533230425401
f1_score=0.8033558169568532


KeyboardInterrupt: 

In [ ]:
test_df = pd.read_csv('test.csv')


In [ ]:
encoded = test_df['text'].apply(lambda x: tokenizer.encode_plus(
    x,
    max_length=64,
    padding='max_length',
    truncation=True,
    return_attention_mask=True
))
test_df['text'].apply(clean_text)

test_df['input_ids'] = encoded.apply(lambda x: x['input_ids'])
test_df['attention_mask'] = encoded.apply(lambda x: x['attention_mask'])

test_dataset = TweetsDataset(test_df, emotions2)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=1
)

In [ ]:
model.eval()

qk = []

with torch.no_grad():
    for idx, batch in enumerate(test_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels']

        preds = model(input_ids, attention_mask).squeeze(0)
        preds = (preds > 0.5)
        preds = preds.int()
        # print(preds)
        # break
        bb = test_df.loc[idx, 'id']

        qk.append([bb] + preds.tolist())

output_df = pd.DataFrame(qk, columns=['id', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])


In [ ]:
output_df.to_csv('pred_eng.csv', index=False)
from google.colab import files
files.download('pred_eng.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
output_df

,id,Anger,Fear,Joy,Sadness,Surprise
0,eng_dev_track_a_00001,1,1,0,0,1
1,eng_dev_track_a_00002,0,0,0,0,0
2,eng_dev_track_a_00003,1,1,0,0,0
3,eng_dev_track_a_00004,0,1,0,0,0
4,eng_dev_track_a_00005,1,1,0,0,1
...,...,...,...,...,...,...
111,eng_dev_track_a_00112,0,1,1,0,0
112,eng_dev_track_a_00113,0,1,0,1,0
113,eng_dev_track_a_00114,0,0,0,1,0
114,eng_dev_track_a_00115,1,1,0,0,1
